In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD585/MD585_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_anchor.txt
File does not exist: 

In [2]:
classifier_id = 30

In [3]:
stack = 'MD658'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [6]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': classifier_id},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
               cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 25.581772 seconds


In [34]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

# downscale = 8
downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v3.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

Resampling scoremaps ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 80.844207 seconds


In [35]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
# viz_downscale = 8
viz_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d -d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v3.py'),
                'stack': stack,
                 'setting': classifier_id,
                'add_label_text': '-a' if add_label_text else '',
                'downscale': viz_downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

visualize scoremaps ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-97ZCW4B5F7TH capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 76.672999 seconds


In [38]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

volume_downscale = 32

if stack in all_alt_nissl_ntb_stacks or stack in all_alt_nissl_tracing_stacks:

    run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d -n' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                    'stack': stack,
                    'setting': classifier_id,
                    'downscale': volume_downscale},
                    kwargs_list=dict(structure=all_known_structures),
                    argument_type='single',
                    cluster_size=16)

else:
    
    run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                    'stack': stack,
                    'setting': classifier_id,
                    'downscale': volume_downscale},
                    kwargs_list=dict(structure=all_known_structures),
                    argument_type='single',
                    cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 116s

constructing score volumes ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 116.025840 seconds


In [44]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)

File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
compute score volume gradients...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 70.765559 seconds


# Global alignment

In [33]:
classifier_setting = 30
warp_setting = 1

In [41]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_num 1' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v3.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD657']),
                argument_type='single',
               cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 261.535937 seconds


In [42]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_idx 0' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                kwargs_list=dict(stack=['MD657']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 60 seconds

transform atlas ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 50.365219 seconds


In [43]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_idx 0' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD657']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 110 seconds

visualize aligned atlas overlay ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 110.714579 seconds


# Local align

In [11]:
classifier_setting = 30
warp_setting = 5

In [12]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

kwargs_list = [dict(stack=stack, structures=shell_escape(json.dumps(all_known_structures_sided[f:l+1])))
for f,l in first_last_tuples_distribute_over(0, len(all_known_structures_sided)-1, 16)]

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d -s %%(structures)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=kwargs_list,                
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2047s for 5 trials.

LOCAL align ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 2047.615233 seconds


In [23]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_idx 0' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                kwargs_list=dict(stack=['MD657']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 60 seconds

transform atlas ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 60.414312 seconds


In [32]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d --trial_idx 0' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD657']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 105 seconds

visualize aligned atlas overlay ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 105.634034 seconds


# Transform locally transformed volumes back to atlas space

In [ ]:
classifier_setting = 2
warp_setting = 3

In [ ]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

# Measure Global Tx Confidence

In [ ]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

# Measure Local Tx Confidence

In [ ]:
warp_setting = 5
classifier_setting = 2

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins